# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 전처리 및 모델 생성]에서만 진행
*   [3. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성


**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 임포트
import tensorflow as tf
from tensorflow import keras

# cifar10 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 클래스 개수 (cifar10 데이터 셋은 10개의 클래스 보유)
num_classes = 10

# **2. 전처리 및 모델 생성**



In [0]:

# 데이터 전처리 (예: normalization)
x_train = x_train / 255.0
x_test = x_test / 255.0

# 순차 모델 생성
model = keras.Sequential()

# Flatten layer: 32 x 32 x 3 image를 3072개의 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))

# 1st hidden layer: fully-connected layer
# (# of inputs = 3072, # of outputs = 1024, actication fuction = relu)
model.add(keras.layers.Dense(1024, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 1024, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(64, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train, y_train, batch_size = 128, epochs = 5, validation_data=[x_test, y_test])

# **3. 모델 저장**

In [0]:
save_path = './saved_model/'
team_name = 'teamXX'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + team_name + '.h5')